# Imports & Data Loading

In [125]:
import numpy as np
import pandas as pd

from functools import reduce

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [145]:
df_2013 = pd.read_csv('../data/2013.csv')
df_2014 = pd.read_csv('../data/2014.csv')
df_2015 = pd.read_csv('../data/2015.csv')
df_2016 = pd.read_csv('../data/2016.csv')
df_2017 = pd.read_csv('../data/2017.csv')
df_2018 = pd.read_csv('../data/2018.csv')
df_2019 = pd.read_csv('../data/2019.csv')
df_2020 = pd.read_csv('../data/2020.csv')
df_2021 = pd.read_csv('../data/2021.csv')
df_2022 = pd.read_csv('../data/2022.csv')


dictionary_df = {
    '2013': df_2013,
    '2014': df_2014,
    '2015': df_2015,
    '2016': df_2016,
    '2017': df_2017,
    '2018': df_2018,
    '2019': df_2019,
    '2020': df_2020,
    '2021': df_2021,
    '2022': df_2022,
}

for key, df in dictionary_df.items():
    dictionary_df[key] = df[(df['Donor Agency'] == 'Bundesministerium fnr Wirtschaftliche Zusammenarbeit und Entwicklung')]
    dictionary_df[key] = dictionary_df[key][dictionary_df[key]['Sector'] != 'Administrative Costs of Donors']
    
    

# Data Processing for Globe Stacked

In [148]:
# Basic Processing of Data

def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby(['Recipient Name']).sum()[['Amount', 'clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['clim_rel_percent'] = grouped_df['clim_rel_amount'] / grouped_df['Amount']
    
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'clim_rel_percent': f"clim_rel_percent_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Recipient Name', how='outer'), processed_grouped_dfs)



## Combined

In [221]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)


# globe_df = globe_df.loc[[142]]

In [222]:
globe_df

,amount_2013,clim_rel_amount_2013,amount_2014,clim_rel_amount_2014,amount_2015,clim_rel_amount_2015,amount_2016,clim_rel_amount_2016,amount_2017,clim_rel_amount_2017,amount_2018,clim_rel_amount_2018,amount_2019,clim_rel_amount_2019,amount_2020,clim_rel_amount_2020,amount_2021,clim_rel_amount_2021,amount_2022,clim_rel_amount_2022
0,87.245749,6.281512,73.771448,5.642177,83.159233,3.442263,86.800258,5.211661,92.559681,1.765870,136.474993,11.051233,150.119022,11.199315,226.390311,7.534234,252.428270,1.536677,265.014798,2.829080
1,153.930790,0.185664,169.047826,0.158667,247.260621,0.160440,239.081905,1.334215,163.615478,0.365663,181.657973,0.213035,276.331320,0.498883,392.548038,0.260622,270.355294,0.561629,237.274430,0.830314
2,110.570602,4.461417,123.680119,5.417927,163.751111,8.053891,202.396396,3.776803,223.352113,4.679552,250.950674,4.491270,289.645386,8.242171,275.831187,8.456473,338.886399,25.023301,336.204882,24.097309
3,104.939416,10.369314,91.545807,3.192433,94.759451,7.356929,100.578689,7.057221,103.391182,8.768037,98.107775,9.659856,89.302996,8.737996,101.871845,6.801880,100.411047,4.537378,107.771939,5.137190
4,95.765407,0.024584,86.811676,3.661154,89.366936,4.462791,87.638400,3.809543,98.101881,2.379202,110.607243,0.475170,129.421547,6.416643,174.007970,16.398422,166.209784,1.498763,174.278158,0.641031
5,158.133749,0.792894,173.291741,0.252810,149.021692,0.730108,186.325695,1.984902,251.097178,2.819138,295.861576,3.016750,316.922164,3.125176,511.053424,9.625516,458.272545,8.225206,1199.102079,22.426779
6,154.994846,1.018042,160.573929,1.003464,159.666195,0.000000,195.485322,0.000000,162.382796,0.130953,151.233217,1.132807,140.140758,1.445219,158.732896,2.483519,172.118568,2.332346,179.901281,3.351427
7,340.989113,102.046666,349.651904,115.889128,342.565770,113.867841,418.833837,150.108114,439.533061,173.489369,468.769446,200.448230,440.633893,202.833941,466.670394,190.186227,490.291625,200.704609,479.942386,190.660330
8,791.300249,56.859578,887.065155,27.507943,801.639790,28.361544,1024.217930,36.227988,964.725825,43.342284,1056.656674,50.659897,1125.044468,56.191150,1140.883682,88.402548,1388.025588,146.060945,1145.696631,111.030814
9,132.299555,0.465190,142.017188,0.390373,155.547184,0.173024,177.827718,0.150417,225.322447,2.239876,294.585502,3.745661,244.152893,11.308853,325.751884,16.244899,396.513655,29.509641,369.056762,26.374849


In [ ]:
years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_rel_amount_col = f'clim_rel_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - globe_df[clim_rel_amount_col]

# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_rel_amount_{year}' for year in years] + 
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')



# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(lambda x: 'Climate Finance' if 'clim_rel_amount' in x else 'Other Funds')

melted_df['Amount'] = melted_df['Amount'] * 1_000_000



melted_df.to_csv("../upload_data/globe_df.csv")

## Split

In [177]:
selected_columns = [col for col in df_merged.columns if col.startswith('amount_') or col.startswith('clim_rel_amount_')\
                   or col.startswith('clim_adapt_') or col.startswith('clim_miti_')]
filtered_df = df_merged[selected_columns]

sums = filtered_df.sum()
globe_df = filtered_df.append(sums, ignore_index=True)
globe_df = globe_df.loc[[142]]


years = range(2013, 2023)

for year in years:
    amount_col = f'amount_{year}'
    clim_adapt_col = f'clim_adapt_amount_{year}'
    clim_miti_col = f'clim_miti_amount_{year}'
    non_clim_col = f'non_clim_amount_{year}'
    globe_df[non_clim_col] = globe_df[amount_col] - (globe_df[clim_adapt_col] + globe_df[clim_miti_col])


# Step 2: Melt the dataframe for plotting
# Including both climatic and non-climatic amounts
melted_df = globe_df.melt(value_vars=[f'clim_adapt_amount_{year}' for year in years] +
                          [f'clim_miti_amount_{year}' for year in years] +
                             [f'non_clim_amount_{year}' for year in years],
                             var_name='Type_Year', value_name='Amount')


# Split 'Type_Year' into separate 'Year' and 'Type' columns
melted_df['Year'] = melted_df['Type_Year'].apply(lambda x: x.split('_')[-1])
melted_df['Type'] = melted_df['Type_Year'].apply(
    lambda x: 'Klimaanpassung Finanzierung' if 'clim_adapt_amount' in x 
    else ('Klimaschutz Finanzierung' if 'clim_miti_amount' in x 
    else 'Andere ODA')
)
melted_df['Amount'] = melted_df['Amounbt'] * 1_000_000




In [178]:
melted_df.to_csv("../upload_data/split_globe.csv")

read_df = pd.read_csv('../upload_data/split_globe.csv')


In [179]:
split_df = pd.read_csv('../upload_data/split_globe.csv')


fig = px.bar(split_df, x='Year', y='Amount', color='Type',
            title='Globale Finanzierungssummen',
            labels={'Amount': 'Finanzierungssumme ($)', 'Year': 'Jahr'},
            category_orders={'Type': ['Andere ODA','Klimaschutz Finanzierung', 'Klimaanpassung Finanzierung']},
            color_discrete_map={'Andere ODA': 'orange', 'Klimaschutz Finanzierung': 'green', 'Klimaanpassung Finanzierung': 'blue'}
            )# This ensures consistent color ordering

fig.update_layout(title_x=0.5)

fig.show()

# Data Processing For Sector Breakdown

In [203]:
def determine_value(row):
    if 2.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 1
    elif row['Climate Adaptation'] == 1.0 and row['Climate Mitigation'] == 1.0:
        return 1
    elif 1.0 in row[['Climate Adaptation', 'Climate Mitigation']].values:
        return 0.5
    else:
        return 0

# final_dict = dictionary_df.copy()    
processed_grouped_dfs = []

    
for year, df in dictionary_df.items():
    
    # Data Cleaning
    use_df = df[['Recipient Name', 'Amount', 'Climate Mitigation', 'Climate Adaptation', 'Sector']]
    use_df['Climate Adaptation'] = use_df['Climate Adaptation'].fillna(0)
    use_df['Climate Mitigation'] = use_df['Climate Mitigation'].fillna(0)
    
    # Applying analysis Rules
    
    use_df['clim_rel'] = use_df.apply(determine_value, axis=1)
    
    use_df['clim_rel_amount'] = use_df['Amount'] * use_df['clim_rel']
    use_df['clim_adapt_amount'] = use_df['Amount'] * (use_df['Climate Adaptation'] / 2)
    use_df['clim_miti_amount'] = use_df['Amount'] * (use_df['Climate Mitigation'] / 2)
    
    
    # Grouping by Country and calculating
    
    grouped_df = use_df.groupby('Sector').sum()[['Amount','clim_rel_amount', 'clim_adapt_amount', 'clim_miti_amount']]
    
    grouped_df['non_miti_adapt'] =grouped_df['Amount'] - (grouped_df['clim_miti_amount'] + grouped_df['clim_adapt_amount'])
    grouped_df['non_clim'] =grouped_df['Amount'] - grouped_df['clim_rel_amount']
    
        
    grouped_df = grouped_df.rename(columns= {
        'Amount': f"amount_{year}",
        'non_miti_adapt': f"non_miti_adapt_{year}",
        'clim_rel_amount': f"clim_rel_amount_{year}",
        'non_clim': f"non_clim_{year}",
        'clim_adapt_amount': f"clim_adapt_amount_{year}",
        'clim_miti_amount': f"clim_miti_amount_{year}"
    })
    
    # Merging DF
    
    processed_grouped_dfs.append(grouped_df)
    



In [204]:
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Sector', how='outer'), processed_grouped_dfs)


In [205]:
df_merged.to_csv('../upload_data/sector_analysis.csv')

df_merged = pd.read_csv('../upload_data/sector_analysis.csv')

In [218]:
list(range(2013,2023))

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [206]:
year = 2013

In [207]:
selected_year = [col for col in df_merged.columns if col.endswith(f'{year}')]
selected_year.append('Sector')
filtered_year_df = df_merged[selected_year]

In [209]:
clim_rel_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_rel_amount_{year}', title='Climate Related Amount by Sector')
clim_rel_fig.show()


In [213]:
non_clim_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'non_clim_{year}', title='Non Climate Related Amount by Sector')
non_clim_fig.show()


In [214]:
clim_adapt_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_adapt_amount_{year}', title='Climate Apaptation Amount by Sector')
clim_adapt_fig.show()


In [215]:
clim_miti_fig = px.sunburst(filtered_year_df, path=['Sector'], values=f'clim_miti_amount_{year}', title='Climate Mitigation Amount by Sector')
clim_miti_fig.show()


In [9]:
dictionary_df['2015'][['Sector', 'SECTOR']]

,Sector,SECTOR
0,IV.1. General Environment Protection,41010
1,IV.1. General Environment Protection,41020
2,IV.2. Other Multisector,43010
3,IV.2. Other Multisector,43010
4,IV.2. Other Multisector,43030
...,...,...
12596,I.1.b. Basic Education,11230
12598,IV.2. Other Multisector,43010
12601,IV.2. Other Multisector,43010
12611,I.5.a. Government & Civil Society-general,15110
